In [1]:
%pip install experta python-constraint

  Using cached experta-1.9.4-py3-none-any.whl.metadata (5.0 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached frozendict-1.2.tar.gz (2.6 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached schema-0.6.7-py2.py3-none-any.whl.metadata (14 kB)
Using cached experta-1.9.4-py3-none-any.whl (35 kB)
Using cached schema-0.6.7-py2.py3-none-any.whl (14 kB)
  Created wheel for frozendict: filename=frozendict-1.2-p

In [2]:
%pip install --upgrade frozendict

  Attempting uninstall: frozendict
    Found existing installation: frozendict 1.2
    Uninstalling frozendict-1.2:
      Successfully uninstalled frozendict-1.2
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
experta 1.9.4 requires frozendict==1.2, but you have frozendict 2.4.6 which is incompatible.


In [2]:
%pip install ortools

  Using cached numpy-2.2.4-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached pandas-2.2.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 10.3 MB/s eta 0:00:00a 0:00:01
Using cached numpy-2.2.4-cp312-cp312-macosx_14_0_arm64.whl (5.2 MB)
Using cached pandas-2.2.3-cp312-cp312-macosx_11_0_arm64.whl (11.4 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
Note: you may need to restart the kernel to use updated packages.


#### Decision Tree Modelling (Working adults)

##### Inputs:

- Age
- Have a car?
- Planning to buy a home?
- Still repaying home loans?
- Have kids?
- Supporting aged parents?
- Monthly take-home pay
  - IF don't know THEN ask for gross salary AND whether there is CPF contribution
- Estimated current expenses on needs / wants / insurance / investment
- Estimated current emergency funds

##### Recommended Actions:

- Needs Analysis:
  - Transportation
  - Food
  - Home loan payment / Rent expense
  - Insurance expense
  - Medical expense
  - Others
  - IF no car AND transport expenditure > $150: RECOMMEND taking public transport instead
  - IF utility bill > 300: RECOMMEND electricity/water saving methods or switching utilities provider 
  - IF insurance expenditure > 15% of take-home salary: REDUCE insurance expenditure
  - IF needs expenditure > 50% of take-home salary: REDUCE spending on needs
    - IF food expense > 15% of take-home salary: REDUCE spending on food
    - IF housing expense > 35% of take-home salary: RECOMMEND finding alternatives
- Wants Analysis:
  - IF wants expenditure > 30% of take-home salary: REDUCE spending on wants
- Savings Analysis:
  - Investments
  - Savings in bank
  - IF investment expenditure < 10% of take-home salary: 
    - IF age < 65: INVEST in long-term investment plans such as ETFs for retirement
    - IF have kids: INVEST in short-term investment plans for kids' tertiary education
    - IF planning to buy a home: INVEST in short-term investment plans for home loan repayment
  - IF monthly savings < 20% of take-home salary: INCREASE savings by REDUCING spending on wants or needs
- Emergency Funds Analysis:
  - IF emergency funds < 3 months of current expenses: INCREASE savings
  - ELSE IF emergency funds < 6 months of current expenses (needs + wants): RECOMMENDED to increase savings


In [ ]:
from experta import KnowledgeEngine, Fact, Rule, MATCH, TEST

class BudgetInfo(Fact):
    """Holds the user's budget-related information."""
    # Basic personal details
    age = int
    number_of_kids = int
    monthly_take_home = float
    planning_to_buy_home = bool
    repaying_home_loans = bool
    supporting_aged_parents = bool
    owns_car = bool

    # Expenses and savings details
    transport_expenditure = float
    food_expenditure = float
    housing_expenditure = float
    insurance_expenditure = float
    other_needs_expenditure = float
    emergency_funds = float
    investment_expenditure = float
    monthly_savings = float


class BudgetAdvisor(KnowledgeEngine):
    # ---------------- Needs Analysis Rules ----------------
    @Rule(BudgetInfo(owns_car=MATCH.owns_car,
                     transport_expenditure=MATCH.transport),
          TEST(lambda owns_car, transport: 
               (not owns_car) and (transport > 150)),
          salience=9)
    def rule_public_transport(self, owns_car, transport):
        print("Your transport spendings are above average. Consider taking public transport instead of using private options.")

    @Rule(BudgetInfo(insurance_expenditure=MATCH.insurance, monthly_take_home=MATCH.take_home),
          TEST(lambda insurance, take_home: insurance > 0.15 * take_home),
          salience=9)
    def rule_reduce_insurance(self, insurance, take_home):
        print("Your spending on insurance is too high! " \
        "The recommended spending on insurance is maximum 15% of your take home-salary.")

    @Rule(BudgetInfo(transport_expenditure=MATCH.transport,
                     food_expenditure=MATCH.food,
                     housing_expenditure=MATCH.housing,
                     insurance_expenditure=MATCH.insurance,
                     other_needs_expenditure=MATCH.others, 
                     monthly_take_home=MATCH.take_home),
          TEST(lambda transport, food, housing, insurance, others, take_home: 
               (transport + food + housing + insurance + others) > 0.5 * take_home),
          salience=10)
    def rule_reduce_needs(self, transport, food, housing, insurance, others, take_home):
        print("Your spending on necessities are too high! " \
        "The recommended spending on needs is maximum 50% of your take-home salary.")

    @Rule(BudgetInfo(food_expenditure=MATCH.food, monthly_take_home=MATCH.take_home),
          TEST(lambda food, take_home: food > 0.15 * take_home),
          salience=9)
    def rule_reduce_food(self, food, take_home):
        print("Your spending on food are too high! " \
        "The recommended spending on food is maximum 15% of your take-home salary.")

    @Rule(BudgetInfo(housing_expenditure=MATCH.housing, monthly_take_home=MATCH.take_home),
          TEST(lambda housing, take_home: housing > 0.35 * take_home),
          salience=9)
    def rule_housing_alternatives(self, housing, take_home):
        print("You may want to consider looking for alternative housing options " \
        "as housing costs are over 35% of your take-home salary.")

    # ---------------- Wants Analysis Rule ----------------
    @Rule(BudgetInfo(transport_expenditure=MATCH.transport,
                     food_expenditure=MATCH.food,
                     housing_expenditure=MATCH.housing,
                     insurance_expenditure=MATCH.insurance,
                     other_needs_expenditure=MATCH.others,
                     monthly_savings=MATCH.savings,
                     monthly_take_home=MATCH.take_home),
          TEST(lambda transport, food, housing, insurance, others, savings, take_home: 
               (take_home - savings - transport - food - housing - insurance - others) > 0.3 * take_home),
          salience=8)
    def rule_reduce_wants(self, transport, food, housing, insurance, others, savings, take_home):
        print("Your spending on wants are too high! " \
        "The recommended spending on wants is maximum 30% of your take-home salary.")

    # ---------------- Savings Analysis Rules ----------------
    @Rule(BudgetInfo(investment_expenditure=MATCH.invest,
                     monthly_take_home=MATCH.take_home,
                     age=MATCH.age,
                     number_of_kids=MATCH.number_of_kids,
                     planning_to_buy_home=MATCH.buy_home),
          TEST(lambda invest, take_home: invest < 0.1 * take_home),
          salience=6)
    def rule_investment_recommendations(self, age, number_of_kids, buy_home):
        if age < 65:
            print("Consider investing in long-term plans (e.g. ETFs) for retirement.")
        if number_of_kids > 0:
            print("Consider short-term investment plans for your kids' tertiary education.")
        if buy_home:
            print("Consider short-term investment plans to help with home loan repayment.")

    @Rule(BudgetInfo(monthly_savings=MATCH.savings, monthly_take_home=MATCH.take_home),
          TEST(lambda savings, take_home: savings < 0.2 * take_home),
          salience=7)
    def rule_increase_savings(self, savings, take_home):
        print("Your savings are too low! You are recommended to save 20% of your take-home salary. " \
        "You can increase your monthly savings by reducing spending on needs or wants.")

    # ---------------- Emergency Funds Analysis Rules ----------------
    @Rule(BudgetInfo(emergency_funds=MATCH.funds, monthly_take_home=MATCH.take_home, monthly_savings=MATCH.savings),
          TEST(lambda funds, take_home, savings: funds < 3 * (take_home - savings)),
          salience=5)
    def rule_emergency_funds_low(self, funds, take_home, savings):
        print("Financial experts recommend maintaining an emergency fund covering 3-6 months of your monthly expenses. " \
        "Currently, your savings fall short of the 3-month minimum. " \
        "To avoid financial stress during unexpected events, please consider increasing your savings as soon as possible.")

    @Rule(BudgetInfo(emergency_funds=MATCH.funds, monthly_take_home=MATCH.take_home, monthly_savings=MATCH.savings),
          TEST(lambda funds, take_home, savings: funds > 3 * (take_home - savings) and funds < 6 * (take_home - savings)),
          salience=5)
    def rule_emergency_funds_medium(self, funds, take_home, savings):
        print("Financial experts recommend maintaining an emergency fund covering 3-6 months of your monthly expenses. " \
        "Build up your emergency funds to cover at least 6 months of your current expenses.")


if __name__ == "__main__":
    engine = BudgetAdvisor()
    engine.reset()

    # Example inputs – these values could be gathered interactively or via a user interface.
    engine.declare(BudgetInfo(
        age=30,
        number_of_kids=1,
        monthly_take_home=4000,
        planning_to_buy_home=True,
        repaying_home_loans=False,
        supporting_aged_parents=False,
        owns_car=True,
        transport_expenditure=100,
        food_expenditure=100,
        housing_expenditure=1000,
        insurance_expenditure=100,
        other_needs_expenditure=700,
        emergency_funds=6000,
        investment_expenditure=300,
        monthly_savings=700
    ))

    engine.run()


Your spending on wants are too high! The recommended spending on wants is maximum 30% of your take-home salary.
Your savings are too low! You are recommended to save 20% of your take-home salary. You can increase your monthly savings by reducing spending on needs or wants.
Consider investing in long-term plans (e.g. ETFs) for retirement.
Consider short-term investment plans for your kids' tertiary education.
Consider short-term investment plans to help with home loan repayment.
Financial experts recommend maintaining an emergency fund covering 3-6 months of your monthly expenses. Currently, your savings fall short of the 3-month minimum. To avoid financial stress during unexpected events, please consider increasing your savings as soon as possible.


In [4]:
from constraint import Problem

def setup_csp(budget):
    """
    Sets up a CSP problem to allocate income among various expense categories.
    :param budget: A dictionary with key 'monthly_take_home' that holds an integer amount.
    :return: List of feasible solutions.
    """
    income = int(budget["monthly_take_home"])
    problem = Problem()
    
    # Define decision variables with a domain from 0 to income (in whole dollars)
    expense_categories = ["savings", "housing", "food", "insurance", "other"]
    for category in expense_categories:
        problem.addVariable(category, range(0, income + 1))
    
    # Add constraint: Sum of all allocations must equal the income.
    def total_allocation(*allocations):
        return sum(allocations) == income
    problem.addConstraint(total_allocation, expense_categories)
    
    # Add ratio constraints based on recommendations:
    # 1. Savings should be at least 20% of income.
    problem.addConstraint(lambda s: s >= 0.2 * income, ("savings",))
    
    # 2. Housing should be no more than 30% of income.
    problem.addConstraint(lambda h: h <= 0.3 * income, ("housing",))
    
    # 3. Food should be no more than 15% of income.
    problem.addConstraint(lambda f: f <= 0.15 * income, ("food",))
    
    # 4. Insurance should be no more than 15% of income.
    problem.addConstraint(lambda i: i <= 0.15 * income, ("insurance",))
    
    # Optionally, other policies (like CPF or debt service constraints) can be added as more constraints.
    
    solutions = problem.getSolutions()
    return solutions

# Example usage:
if __name__ == "__main__":
    budget_example = {"monthly_take_home": 4000}
    solutions = setup_csp(budget_example)
    
    # For demonstration, show the first solution found (if one exists)
    if solutions:
        print("Number of feasible allocation solutions found:", len(solutions))
        print("Example solution:")
        print(solutions[0])
    else:
        print("No feasible allocation found with the given constraints.")


KeyboardInterrupt: 

In [3]:
from constraint import Problem
from typing import Dict, Any

def setup_csp_with_dynamic_constraints(budget: Dict[str, Any], custom_constraints: Dict[str, float]):
    """
    Sets up a CSP problem to allocate income among expense categories with some constraints dynamically defined by the user.
    
    Args:
      budget: A dictionary with 'monthly_take_home' as a key.
      custom_constraints: A dictionary for dynamic constraints.
          e.g., {"savings_min": 0.3, "food_max": 0.12} meaning 
            - savings should be at least 30% of income
            - food expenditure should be at most 12% of income
    Returns:
      A list of feasible solutions.
    """
    income = int(budget["monthly_take_home"])
    problem = Problem()
    
    # Define expense categories
    expense_categories = ["savings", "housing", "food", "insurance", "other"]
    for category in expense_categories:
        # We use the full range [0, income] assuming whole-dollar increments.
        problem.addVariable(category, range(0, income + 1, 50))
    
    # Constraint: The sum of all allocations must equal the monthly income
    def total_allocation(*allocations):
        return sum(allocations) == income
    problem.addConstraint(total_allocation, expense_categories)
    
    # Static ratio constraints (you might retain some default values)
    problem.addConstraint(lambda s: s >= 0.2 * income, ("savings",))
    problem.addConstraint(lambda h: h <= 0.3 * income, ("housing",))
    problem.addConstraint(lambda f: f <= 0.15 * income, ("food",))
    problem.addConstraint(lambda i: i <= 0.15 * income, ("insurance",))
    
    # Dynamic constraints based on user's definition
    # Example: "savings_min" = 0.3 means savings must be at least 30% of income.
    if "savings_min" in custom_constraints:
        savings_min = custom_constraints["savings_min"]
        problem.addConstraint(lambda s: s >= savings_min * income, ("savings",))
    
    # Example: "food_max" = 0.12 means food cost must be at most 12% of income.
    if "food_max" in custom_constraints:
        food_max = custom_constraints["food_max"]
        problem.addConstraint(lambda f: f <= food_max * income, ("food",))
    
    # Additional dynamic constraints can be added similarly.
    # For example, you might let the user define a custom limit for "other" expenses:
    if "other_max" in custom_constraints:
        other_max = custom_constraints["other_max"]
        problem.addConstraint(lambda o: o <= other_max * income, ("other",))
    
    solutions = problem.getSolutions()
    return solutions


# Example usage:
if __name__ == "__main__":
    # Budget input from the user.
    budget_example = {"monthly_take_home": 4000}
    
    # Custom constraints defined by the user (e.g., through the web interface).
    # Here, the user wants savings to be at least 30% and food to be at most 12% of income.
    custom_constraints = {
        "savings_min": 0.9,  # 30% minimum savings
        "food_max": 0.12     # 12% maximum for food expenses
    }
    
    solutions = setup_csp_with_dynamic_constraints(budget_example, custom_constraints)
    
    if solutions:
        print("Number of feasible allocation solutions found:", len(solutions))
        print("Example solution:")
        print(solutions[0])
    else:
        print("No feasible allocation found with the given dynamic constraints.")


Number of feasible allocation solutions found: 495
Example solution:
{'savings': 4000, 'food': 0, 'insurance': 0, 'housing': 0, 'other': 0}


In [8]:
from ortools.sat.python import cp_model

def optimize_budget_with_min_deviation(current_allocations, income):
    """
    Optimize budget allocation while minimizing deviation from current allocations.
    
    Arguments:
      current_allocations: dict with keys corresponding to categories (e.g., 'savings', 'housing', etc.)
                           and current values.
      income: total monthly take-home income.
    
    Returns:
      An optimized allocation that minimizes deviation.
    """
    model = cp_model.CpModel()
    
    # Define budget categories
    categories = ['transport', 'food', 'housing', 'insurance', 'investment', 'savings', 'total_needs', 'total_wants']
    variables = {}
    deviations = {}
    
    # Suppose domain is in whole dollars.
    for cat in categories:
        # Decision variable for each category: allocation from 0 to income.
        variables[cat] = model.NewIntVar(0, income, cat)
        # Deviation variable (absolute difference) for each category.
        max_deviation = income  # Worst-case deviation can be as high as income.
        deviations[cat] = model.NewIntVar(0, max_deviation, f"dev_{cat}")
        
        # Add constraint linking deviation with the absolute difference.
        # model.AddAbsEquality(deviations[cat], variables[cat] - current_allocations[cat])
        # Since CP-SAT does not have a built-in absolute value in older versions,
        # we can model it using:
        diff = model.NewIntVar(-income, income, f"diff_{cat}")
        dev = (variables[cat] - current_allocations[cat]) ** 2
        model.Add(diff == dev)
        # model.Add(diff == (variables[cat] - current_allocations[cat])**2)
        model.AddAbsEquality(deviations[cat], diff)
    
    # Total needs allocation constraint: sum of needs must be less than or equal to total_needs.
    model.Add(variables['total_needs'] <= sum(variables[cat] for cat in ['transport', 'food', 'housing', 'insurance']))

    # Total allocation constraint: sum of allocations must equal income.
    model.Add(sum(variables[cat] for cat in ['total_needs', 'total_wants', 'savings']) == income)
    
    # Add hard constraints as needed:
    model.Add(variables['savings'] >= int(0.2 * income))
    model.Add(variables['total_needs'] >= int(0.5 * income))
    model.Add(variables['total_wants'] >= int(0.3 * income))
    
    # Objective: minimize total deviation
    model.Minimize(sum(deviations[cat] for cat in categories))
    
    # Solve the model.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    
    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        result = {cat: solver.Value(variables[cat]) for cat in categories}
        total_deviation = sum(solver.Value(deviations[cat]) for cat in categories)
        return result, total_deviation
    else:
        return None, None

# Example usage:
if __name__ == "__main__":
    # Assume current_allocations is what the user currently spends
    current_allocations = {
        'transport': 150,
        'food': 1000,
        'housing': 1200,
        'insurance': 700,
        'investment': 300,
        'savings': 200,
        'total_needs': 1000,
        'total_wants': 100,  # other expenditures.
    }
    income = 4000
    solution, deviation = optimize_budget_with_min_deviation(current_allocations, income)
    if solution:
        print("Optimized Allocation:", solution)
        print("Total Deviation:", deviation)
    else:
        print("No feasible solution found.")


NotImplementedError: calling ** on a linear expression is not supported, please use CpModel.add_multiplication_equality